In [ ]:
#importing all functions
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

In [10]:
from pyspark.sql import functions as fn

## Understanding the Dataset

In [ ]:
#reading the dataset 
trails_df = spark.read.json('dataset.json',multiLine=True)

In [ ]:
#displaying the content of the dataframe limiting to 5 records
trails_df.show(5)

In [33]:
#structure of the datset
trails_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [34]:
# displaying the columns in the dataframe
trails_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

## Flattening the json object(nested cloumns) within the structure
- maintaining same hierachy as clinician for branch,name and role
- maintaining same hierachy as result for conclusion and sideeffectsonparticipant

In [11]:
columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [35]:
trails_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [16]:
#checking null values by counting null values per column
trails_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()



+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



## Cleaning

In [ ]:
#flatten  df
# address null values
# renaming


In [17]:
#creating a copy of the dataframe
new_trials_df = trails_df.select(columns)

In [18]:
#checking the structure of the new df
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [24]:
#renaming columns using withColumnsRenamed method
new_trials_df = new_trials_df.withColumnsRenamed(
    {'branch' : 'clinic_branch','name': 'head_clinician','role':'clinician_role',
    'experimentenddate' : 'experiment_end_date','experimentstartdate': 'experiment_start_date',
    'noofhourspassedatfirstreaction' : 'hours_passed_at_first_reaction',
    'conclusion' : 'experiment_conclusion','sideeffectsonparticipant':'patient_side_effect'
    })

In [25]:
#checking new df
new_trials_df.show()

+----------------+----------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+---------------------+-------------------+
|ageofparticipant|clinician_branch|clinician_name|clinician_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|experiment_conclusion|patient_side_effect|
+----------------+----------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+---------------------+-------------------+
|              19|         Ontario|          Saul|     therapist|  Placebo|      1619827200000|        1617235200000|                            52|        BP normalized|     rashes on neck|
|              14|         Ontario|          Saul|         nurse| Naproxen|      1619827200000|        1617235200000|                            78|            Follow-up|                N/A|
|              17|         Ontario|          

# Working all Null Values

In [36]:
# working on null values
new_trials_df.describe().show()

+-------+------------------+----------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+---------------------+-------------------+
|summary|  ageofparticipant|clinician_branch|clinician_name|clinician_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|experiment_conclusion|patient_side_effect|
+-------+------------------+----------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+---------------------+-------------------+
|  count|              3586|            3586|          3586|          3586|     3586|                3586|                 3586|                          3513|                 3586|               3586|
|   mean|17.507250418293363|            null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|                 null|           

In [37]:
#assigning values to null values seen in multiple columns
new_trials_df = new_trials_df.na.fill({'experiment_conclusion': 'unknown','clinician_role' : 'unknown'})


In [38]:
new_trials_df.describe().show()

+-------+------------------+----------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+---------------------+-------------------+
|summary|  ageofparticipant|clinician_branch|clinician_name|clinician_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|experiment_conclusion|patient_side_effect|
+-------+------------------+----------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+---------------------+-------------------+
|  count|              3586|            3586|          3586|          3586|     3586|                3586|                 3586|                          3513|                 3586|               3586|
|   mean|17.507250418293363|            null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|                 null|           